# Build a Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents and having more finer control, LangChain recommends to use LangGraph, which we cover in other courses.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

![](https://i.imgur.com/1uVnBAm.png)



## Install OpenAI, and LangChain dependencies

In [1]:
# !pip install langchain==0.3.14
# !pip install langchain-openai==0.3.0
# !pip install langchain-community==0.3.14

In [2]:
# !pip install markitdown

## Enter Open AI API Key

In [3]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [4]:
# TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [5]:
# WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

## Setup Environment Variables

In [6]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY
# os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [ ]:
# =============================================================================
# LOAD ENVIRONMENT VARIABLES
# =============================================================================
# This loads API keys from a .env file in the project directory.
# Your .env file should contain:
#   OPENAI_API_KEY=your_openai_key
#   TAVILY_API_KEY=your_tavily_key  
#   WEATHER_API_KEY=your_openweathermap_key
# =============================================================================

from dotenv import load_dotenv, find_dotenv

# find_dotenv() searches for .env file in current and parent directories
load_dotenv(find_dotenv())

True

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/TyPAYXE.png)

In [ ]:
# =============================================================================
# TOOL CREATION SECTION
# =============================================================================
# In LangChain, "tools" are functions that agents can use to interact with 
# external systems (APIs, databases, web search, etc.).
# 
# The @tool decorator converts a regular Python function into a LangChain-
# compatible tool that can be used by agents for decision-making.
# =============================================================================

# --- Import Required Libraries ---
from langchain_core.tools import tool  # Decorator to create tools from functions
from markitdown import MarkItDown  # Converts web content to markdown format
from langchain_community.tools.tavily_search import TavilySearchResults  # Web search API
from tqdm import tqdm  # Progress bar for visual feedback during processing
from concurrent.futures import ThreadPoolExecutor, TimeoutError  # Parallel processing
import requests
import os
from warnings import filterwarnings
filterwarnings('ignore')

# =============================================================================
# TOOL 1: Web Search and Information Extraction Tool
# =============================================================================
# This tool uses Tavily API for web search and MarkItDown for content extraction

# Initialize Tavily search with advanced settings
tavily_tool = TavilySearchResults(
    max_results=5,           # Return top 5 search results
    search_depth='advanced', # Use advanced search for better quality results
    include_answer=False,    # Don't include AI-generated summary
    include_raw_content=True # Include raw HTML content from pages
)

# Configure HTTP session with browser-like headers
# This prevents websites from blocking our requests (anti-bot protection bypass)
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})

# Initialize MarkItDown for converting web pages to readable markdown text
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """
    Search the web for a query and extract useful information from the search links.
    
    This tool performs two main operations:
    1. Searches the web using Tavily API to find relevant URLs
    2. Extracts and converts content from each URL to readable text
    
    Args:
        query (str): The search query to find information about
        
    Returns:
        list: A list of extracted text content from relevant web pages
    """
    print('Calling web search tool')
    
    # Step 1: Get search results from Tavily
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract and format content from a single URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content
    
    # Step 2: Process URLs in parallel for faster execution
    # ThreadPoolExecutor allows concurrent URL fetching
    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Set 15-second timeout to prevent hanging on slow websites
                content = future.result(timeout=15)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


# =============================================================================
# TOOL 2: Weather Information Tool  
# =============================================================================
# This tool fetches real-time weather data using OpenWeatherMap API

@tool
def get_weather(query: str) -> dict:
    """
    Fetch current weather data for a specified location using OpenWeatherMap API.
    
    Args:
        query (str): The city name to get weather information for
        
    Returns:
        dict: Weather data including temperature, humidity, wind speed, etc.
              Returns "Weather Data Not Found" if the location is invalid
    """
    # Construct API URL with city name and API key
    # Note: The ',IN' suffix restricts search to India - modify as needed
    url = f"https://api.openweathermap.org/data/2.5/weather?q={query},IN&appid={os.getenv('WEATHER_API_KEY')}&units=metric"

    response = requests.get(url)
    data = response.json()
    
    # Validate response by checking if 'name' field exists
    if data.get("name"):
        return data
    else:
        return "Weather Data Not Found"

## Test Tool Calling with LLM

In [ ]:
# =============================================================================
# BINDING TOOLS TO THE LLM
# =============================================================================
# Before creating an agent, let's understand how LLMs interact with tools.
# The .bind_tools() method tells the LLM about available tools and their schemas.
# This allows the LLM to decide WHEN to use a tool and WHAT arguments to pass.
# =============================================================================

from langchain_openai import ChatOpenAI

# Initialize ChatGPT with temperature=0 for deterministic responses
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

# Create a list of tools the agent can use
tools = [search_web_extract_info, get_weather]

# Bind tools to the LLM - this creates tool-aware version of the model
# The LLM can now "call" these tools when it determines they're needed
chatgpt_with_tools = chatgpt.bind_tools(tools)

In [ ]:
# =============================================================================
# TEST 1: LLM Decides to Use Web Search Tool
# =============================================================================
# When we ask about real-time information (like earnings calls), the LLM
# recognizes it needs external data and decides to call the search tool.

prompt = "Get details of Microsoft's earnings call Q4 2024"
response = chatgpt_with_tools.invoke(prompt)

# The response contains 'tool_calls' - these are the tools the LLM wants to use
# Notice how the LLM automatically formats the query for the search tool
response.tool_calls

[{'name': 'search_web_extract_info',
  'args': {'query': 'Microsoft earnings call Q4 2024 details'},
  'id': 'call_hl9dhRJt5dgIPOSvTEe4i4oO',
  'type': 'tool_call'}]

In [ ]:
# =============================================================================
# TEST 2: LLM Decides to Use Weather Tool
# =============================================================================
# For weather-related queries, the LLM chooses the get_weather tool instead.
# This demonstrates intelligent tool selection based on query context.

prompt = "how is the weather in Bangalore today"
response = chatgpt_with_tools.invoke(prompt)

# The LLM has selected get_weather and extracted "Bangalore" as the location
response.tool_calls

[{'name': 'get_weather',
  'args': {'query': 'Bangalore'},
  'id': 'call_PlNKoxc2iB93w2stVrhVQ0yS',
  'type': 'tool_call'}]

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [ ]:
# =============================================================================
# CREATING THE AGENT PROMPT TEMPLATE (ReAct Pattern)
# =============================================================================
# The prompt template defines how the agent should think and behave.
# We use the ReAct (Reasoning + Acting) pattern which structures the agent's
# thought process into: Thought → Action → Observation → (repeat) → Answer
# =============================================================================

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the system prompt that guides agent behavior
# This prompt implements the ReAct pattern for structured reasoning
SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

# Build the prompt template with multiple components:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),  # System instructions for agent behavior
        MessagesPlaceholder(variable_name="history", optional=True),  # Chat history (for memory)
        ("human", "{query}"),  # User's input query
        MessagesPlaceholder(variable_name="agent_scratchpad"),  # Agent's working memory
    ]
)

# Display the prompt structure
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [ ]:
# =============================================================================
# CREATE THE AGENT
# =============================================================================
# The agent is the "brain" that decides what actions to take based on input.
# It uses the LLM to reason about the problem and determine which tools to use.
# 
# IMPORTANT: The agent itself does NOT execute tools - it only DECIDES what to do.
# The AgentExecutor (next step) handles the actual tool execution.
# =============================================================================

from langchain.agents import create_tool_calling_agent

# Re-initialize the LLM (we use a fresh instance for the agent)
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)

# Define the tools available to this agent
tools = [search_web_extract_info, get_weather]

# Create the agent by combining: LLM + Tools + Prompt Template
# Note: We pass 'chatgpt' not 'chatgpt_with_tools' because
# create_tool_calling_agent calls .bind_tools() internally
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [ ]:
# =============================================================================
# CREATE THE AGENT EXECUTOR
# =============================================================================
# The AgentExecutor is the runtime that:
# 1. Takes user input and passes it to the agent
# 2. Executes the tools that the agent decides to use
# 3. Passes tool results back to the agent
# 4. Repeats until the agent has a final answer
#
# Think of it as: Agent = Brain, AgentExecutor = Body that takes action
# =============================================================================

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,                      # The agent (decision maker)
    tools=tools,                      # Available tools to execute
    early_stopping_method='force',    # Force stop if max iterations reached
    max_iterations=10                 # Maximum reasoning loops (prevents infinite loops)
)

In [ ]:
# =============================================================================
# COMPARISON: LLM WITHOUT Agent (No Tools)
# =============================================================================
# First, let's see what happens when we ask the LLM directly WITHOUT using
# the agent. The LLM can only use its training data (which has a cutoff date)
# and cannot access real-time information.
# =============================================================================

query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)

# Notice: The LLM admits it doesn't have current data - it cannot search the web!
print(response.content)

As of my last update, Nvidia's Q4 2024 earnings call has not occurred, as my data only goes up to October 2023. Therefore, I cannot provide specific details about the call. However, in a typical Nvidia earnings call, you can expect discussions on the following key points:

1. **Financial Performance**: Overview of revenue, net income, and earnings per share compared to previous quarters and analyst expectations.

2. **Segment Performance**: Insights into the performance of different business segments, such as gaming, data center, professional visualization, and automotive.

3. **Market Trends**: Commentary on market conditions affecting Nvidia's business, including demand for GPUs, AI, and data center products.

4. **Product Updates**: Announcements or updates on new products, technologies, or partnerships.

5. **Guidance**: Forward-looking statements regarding expected financial performance and strategic priorities for the upcoming quarters.

6. **Challenges and Risks**: Discussion of

In [ ]:
# Helper import for rendering markdown output nicely in Jupyter
from IPython.display import display, Markdown

In [ ]:
# =============================================================================
# COMPARISON: LLM WITH Agent (Has Access to Tools)
# =============================================================================
# Now let's ask the SAME question using the agent. The agent will:
# 1. Recognize it needs real-time data
# 2. Call the search_web_extract_info tool
# 3. Process the results and provide an accurate answer
# =============================================================================

query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})

# The agent successfully retrieves and summarizes real information!
display(Markdown(response['output']))

Calling web search tool


 20%|██        | 1/5 [00:00<00:01,  2.61it/s]Cannot set gray non-stroke color because /'P25' is an invalid float value
Cannot set gray non-stroke color because /'P46' is an invalid float value
Cannot set gray non-stroke color because /'P137' is an invalid float value
Cannot set gray non-stroke color because /'P172' is an invalid float value
Cannot set gray non-stroke color because /'P205' is an invalid float value
Cannot set gray non-stroke color because /'P238' is an invalid float value
Cannot set gray non-stroke color because /'P271' is an invalid float value
Cannot set gray non-stroke color because /'P306' is an invalid float value
Cannot set gray non-stroke color because /'P339' is an invalid float value
Cannot set gray non-stroke color because /'P374' is an invalid float value
Cannot set gray non-stroke color because /'P442' is an invalid float value
 40%|████      | 2/5 [00:02<00:04,  1.44s/it]

Error extracting from url: https://s201.q4cdn.com/141608511/files/doc_financials/2024/q4/NVDA-F4Q24-Quarterly-Presentation-FINAL.pdf - 'NoneType' object has no attribute 'strip'


 60%|██████    | 3/5 [00:03<00:02,  1.30s/it]

Error extracting from url: https://www.youtube.com/watch?v=hV6WxM3S80g - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:06<00:00,  1.24s/it]

Error extracting from url: https://nvidianews.nvidia.com/_gallery/download_pdf/65d669a33d63329bbf62672a/ - 'NoneType' object has no attribute 'strip'


Nvidia's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, a 22% increase from the previous quarter and a 265% rise from the same period a year ago. The fiscal year 2024's revenue reached $60.9 billion, up by 126%. The GAAP EPS was $4.93, up 765% year-over-year, and Non-GAAP EPS was $5.16, reflecting a 486% increase.

2. **Data Center and AI**: Approximately 40% of Nvidia's data center revenue, which hit a record $18.4 billion in Q4, was attributed to AI inference. Nvidia's AI enterprise software reached an annualized revenue run rate of $1 billion, indicating potential growth in leveraging AI across various industries.

3. **Stock Performance**: Following the earnings announcement, Nvidia's stock surged by more than 8% in after-hours trading, reflecting positive market sentiment and investor confidence.

4. **Strategic Initiatives**: Nvidia's strategic focus on AI and accelerated computing, along with its expansive ecosystem of partnerships, positions it well for sustained growth. Despite regulatory challenges in China, Nvidia adapted by offering alternative products, mitigating potential impacts on its business.

5. **China Revenue**: Nvidia's data center revenue in China declined significantly due to U.S. government export control regulations. However, China represented a single-digit percentage of data center revenue for Q4, and Nvidia has started shipping alternatives that don't require a license for the China market.

Overall, Nvidia's Q4 2024 earnings call underscored its strong financial performance, strategic focus on AI, and ability to navigate regulatory challenges, positioning it for continued growth and innovation.

In [ ]:
# =============================================================================
# TEST: Another Web Search Query
# =============================================================================
# Let's test with another company to verify the agent works consistently

query = """Summarize the key points discussed in Intel's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


 40%|████      | 2/5 [00:01<00:02,  1.40it/s]

Error extracting from url: https://morethanmoore.substack.com/p/intel-2024-q4-financials - 'NoneType' object has no attribute 'strip'


 60%|██████    | 3/5 [00:02<00:01,  1.06it/s]

Error extracting from url: https://www.fool.com/earnings/call-transcripts/2025/01/30/intel-intc-q4-2024-earnings-call-transcript/ - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:05<00:00,  1.20s/it]

Error extracting from url: https://newsroom.intel.com/corporate/intel-reports-fourth-quarter-full-year-2024-financial-results - 'NoneType' object has no attribute 'strip'


Intel's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Intel reported Q4 revenue of $14.3 billion, up 7% sequentially, with a non-GAAP gross margin of 42.1%, surpassing guidance. Earnings per share (EPS) were $0.13, slightly above the expected $0.12. However, the full-year 2024 revenue was $53.1 billion, down 2.1% year-over-year, with a negative EPS of $0.13.

2. **Challenges and Guidance**: Despite surpassing guidance, Intel faces increased competition in the AI PC category and is not yet significantly participating in the cloud-based AI data center market. The company reported a significant operating loss in its foundry services and expects a sequential revenue decline in Q1 2025 due to macroeconomic uncertainty and seasonality.

3. **Strategic Developments**: Intel remains a leader in AI PC CPUs and plans to ship over 100 million systems by the end of 2025. The launch of Panther Lake on Intel 18A is on track for the second half of 2025. Intel Foundry Services is focusing on building a world-class foundry, aligning with US government interests.

4. **Leadership Changes**: The company appointed two interim co-CEOs, David Zinsner and Michelle Johnston Holthaus, following the departure of CEO Pat Gelsinger.

5. **Product and Market Strategy**: Intel is focusing on its Jaguar Shores product for the AI data center market and has decided not to sell its Falcon Shores AI processor for servers, using it instead as a test chip.

Overall, while Intel exceeded expectations for Q4 2024, it faces ongoing challenges in market competition and strategic execution.

In [ ]:
# =============================================================================
# DEMONSTRATING THE MEMORY LIMITATION
# =============================================================================
# This query asks "which company" - referring to previous queries about
# Nvidia and Intel. However, the agent has NO MEMORY of previous conversations!
# It will ask for clarification because it doesn't remember the context.
# (We'll address this limitation with session-based memory in future lessons)

query = """which company's future outlook looks to be better?"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

To provide an accurate assessment of a company's future outlook, I would need to know which specific companies you are interested in comparing. Please provide the names of the companies you want to evaluate, and I can help gather relevant information to assess their future prospects.

In [ ]:
# =============================================================================
# TEST: Weather Tool - Indian City (Bangalore)
# =============================================================================
# The agent should recognize this as a weather query and use the get_weather tool.
# Our weather tool is configured for Indian cities (has ,IN suffix in API call).

query = """how is the weather in Bangalore today? show detailed statistics"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

The weather in Bangalore today is as follows:

- **Condition**: Mist
- **Temperature**: 14.84°C
- **Feels Like**: 14.76°C
- **Minimum Temperature**: 14.45°C
- **Maximum Temperature**: 15.28°C
- **Pressure**: 1015 hPa
- **Humidity**: 91%
- **Visibility**: 1800 meters
- **Wind Speed**: 5.36 m/s, coming from 74° with gusts up to 12.96 m/s
- **Cloudiness**: 20%

Sunrise is at 05:43 AM and sunset will be at 05:51 PM local time.

In [ ]:
# =============================================================================
# TEST: Weather Tool - Non-Indian City (Dubai) - LIMITATION DEMO
# =============================================================================
# Dubai is NOT in India, so our weather API (configured with ,IN suffix) won't
# find it. The agent will fall back to web search, which may or may not work.
# This demonstrates the importance of proper tool configuration!

query = """how is the weather in Dubai today? show detailed statistics"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


 20%|██        | 1/5 [00:00<00:01,  3.32it/s]

Error extracting from url: https://en.climate-data.org/asia/united-arab-emirates/dubai/dubai-705/t/december-12/ - 499 Client Error: <none> for url: https://en.climate-data.org/asia/united-arab-emirates/dubai/dubai-705/t/december-12/


 40%|████      | 2/5 [00:15<00:26,  8.95s/it]

Extraction timed out for url: https://www.makemytrip.com/tripideas/dubai/dubai-weather-in-december


...

 80%|████████  | 4/5 [00:18<00:03,  3.92s/it]

Error extracting from url: https://www.khaleejtimes.com/uae/weather/december-13-2025 - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:18<00:00,  3.70s/it]

Error extracting from url: https://gulfnews.com/uae/weather/uae-authorities-warn-of-intense-rain-and-rough-seas-1.500378825 - 'NoneType' object has no attribute 'strip'


I was unable to retrieve the current weather details for Dubai. You might want to check a reliable weather website or app for the most up-to-date information.

In [ ]:
# =============================================================================
# MEMORY LIMITATION DEMO (Again)
# =============================================================================
# Again, the agent doesn't remember previous queries about Bangalore and Dubai.
# It will ask for clarification about which cities we're comparing.

query = """which city is hotter?"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

To determine which city is hotter, I need to know the names of the cities you are comparing. Could you please provide the names of the cities?

The agent is doing pretty well but unfortunately it doesn't remember conversations. We will use some user-session based memory to store this and dive deeper into this in the next video.

## Bonus: Alternative Approach - Manual Tool Selection with LLMChain

The following section demonstrates an **alternative approach** to building agents using manual LLMChain-based tool selection. This approach gives you more control over the tool selection process but requires more code.

**Key Differences from AgentExecutor:**
- More explicit control over the tool selection logic
- Uses separate chains for: tool selection → input formatting → response generation
- Good for understanding how agents work under the hood

> ⚠️ **Note:** This is a simplified educational example using mock tools. The first example uses real API tools.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

# Create an LLM chain for deciding which tool to use
tool_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a tool selection assistant. 
    Based on the user's question, determine which tool would be most appropriate.
    Respond with only the tool name: "search_web_extract_info", "get_weather" .
    If none of these tools can help, respond with "None"."""),
    ("human", "{query}")
])

tool_selection_chain = LLMChain(
    llm=chatgpt,
    prompt=tool_selection_prompt,
    output_key="selected_tool"
)

# Create an LLM chain for formatting tool inputs
tool_input_prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract the specific input needed for the tool from the user's query.
    For WeatherService: Extract the location name.
    for get_weather extract the keywords and search the info in web
    Provide only the extracted information, nothing else."""),
    ("human", "Tool to use: {selected_tool}\nUser query: {query}")
])

tool_input_chain = LLMChain(
    llm=chatgpt,
    prompt=tool_input_prompt,
    output_key="tool_input"
)

# Create an LLM chain for formatting the final response
response_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a helpful response to the user's query using the tool's output."),
    ("human", "User query: {query}\nTool used: {selected_tool}\nTool output: {tool_output}")
])

response_chain = LLMChain(
    llm=chatgpt,
    prompt=response_prompt,
    output_key="response"
)

# Function to run the full chain
def process_query(query: str) -> str:
    # First, select which tool to use
    tool_selection_result = tool_selection_chain.invoke({"query": query})
    selected_tool = tool_selection_result["selected_tool"].strip()
    print(f"Selected tool: {selected_tool}")
    
    if selected_tool == "None":
        return "I don't have a tool that can help with this query."
    
    # Next, format the input for the selected tool
    tool_input_result = tool_input_chain.invoke({
        "selected_tool": selected_tool,
        "query": query
    })
    tool_input = tool_input_result["tool_input"].strip()
    print(f"Tool input: {tool_input}")
    
    # Find and run the appropriate tool
    tool_output = "Tool not found"
    for tool in tools:
        if tool.name == selected_tool:
            tool_output = tool.func(tool_input)
            break
    print(f"Tool output: {tool_output}")
    
    # Format the final response
    response_result = response_chain.invoke({
        "query": query,
        "selected_tool": selected_tool,
        "tool_output": tool_output
    })
    return response_result["response"]

examples = [
        "What is 234 * 78.5?",
        "What's the weather like in Bangalore?",
        "Tell me about LangChain.",
    ]
    
for example in examples:
    print(f"\n\nQUERY: {example}")
    print("-" * 50)
    response = process_query(example)
    print("FINAL RESPONSE:")
    print(response)

In [ ]:
# =============================================================================
# ALTERNATIVE APPROACH: Manual Tool Selection with LLMChain
# =============================================================================
# This example demonstrates how to build a simple agent-like system from scratch
# using LLMChain instead of the built-in AgentExecutor.
#
# WORKFLOW:
# 1. Tool Selection Chain → LLM decides which tool to use
# 2. Tool Input Chain → LLM extracts the right input for the tool
# 3. Tool Execution → Run the selected tool
# 4. Response Chain → LLM formats the final response
#
# This approach gives you fine-grained control but requires more code.
# Use AgentExecutor for production; use this for learning!
# =============================================================================

import os
from typing import Dict, Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, create_structured_chat_agent

# =============================================================================
# STEP 1: Define Mock Tool Functions
# =============================================================================
# These are simple mock implementations for demonstration purposes

def calculator(expression: str) -> float:
    """Evaluate a mathematical expression safely."""
    try:
        return eval(expression)  # Note: eval() is unsafe in production!
    except Exception as e:
        return f"Error: {str(e)}"

def get_weather_mock(location: str) -> str:
    """Mock weather function - returns fake data for demo."""
    return f"The weather in {location} is sunny with a temperature of 72°F."

def search_database(query: str) -> str:
    """Search a mock database for information."""
    # Simple key-value database for demonstration
    database = {
        "python": "Python is a high-level programming language.",
        "langchain": "LangChain is a framework for LLM applications.",
        "llm": "LLM stands for Large Language Model."
    }
    query = query.lower()
    return database.get(query, f"No information found for '{query}'.")

# =============================================================================
# STEP 2: Create Tool Objects
# =============================================================================
# Wrap our functions in LangChain Tool objects with names and descriptions

tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Useful for performing mathematical calculations"
    ),
    Tool(
        name="WeatherService",
        func=get_weather_mock,
        description="Get current weather for a location"
    ),
    Tool(
        name="Database",
        func=search_database,
        description="Search the database for information"
    )
]

# =============================================================================
# STEP 3: Initialize the LLM
# =============================================================================
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# =============================================================================
# STEP 4: Create the Tool Selection Chain
# =============================================================================
# This chain asks the LLM to decide which tool is most appropriate

tool_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a tool selection assistant. 
    Based on the user's question, determine which tool would be most appropriate.
    Respond with only the tool name: "Calculator", "WeatherService", or "Database".
    If none of these tools can help, respond with "None"."""),
    ("human", "{query}")
])

tool_selection_chain = LLMChain(
    llm=llm,
    prompt=tool_selection_prompt,
    output_key="selected_tool"
)

# =============================================================================
# STEP 5: Create the Tool Input Extraction Chain
# =============================================================================
# This chain extracts the specific input needed for the selected tool

tool_input_prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract the specific input needed for the tool from the user's query.
    For Calculator: Extract the mathematical expression.
    For WeatherService: Extract the location name.
    For Database: Extract the search query.
    Provide only the extracted information, nothing else."""),
    ("human", "Tool to use: {selected_tool}\nUser query: {query}")
])

tool_input_chain = LLMChain(
    llm=llm,
    prompt=tool_input_prompt,
    output_key="tool_input"
)

# =============================================================================
# STEP 6: Create the Response Formatting Chain
# =============================================================================
# This chain takes the tool output and creates a user-friendly response

response_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a helpful response to the user's query using the tool's output."),
    ("human", "User query: {query}\nTool used: {selected_tool}\nTool output: {tool_output}")
])

response_chain = LLMChain(
    llm=llm,
    prompt=response_prompt,
    output_key="response"
)

# =============================================================================
# STEP 7: Create the Main Processing Function
# =============================================================================
# This function orchestrates the entire workflow, similar to what AgentExecutor
# does automatically. This manual approach gives you visibility into each step.

def process_query(query: str) -> str:
    """
    Process a user query using the manual tool selection approach.
    
    Steps:
    1. Select appropriate tool
    2. Extract tool input from query
    3. Execute the tool
    4. Format and return response
    """
    
    # STEP 7a: Ask LLM to select which tool to use
    tool_selection_result = tool_selection_chain.invoke({"query": query})
    selected_tool = tool_selection_result["selected_tool"].strip()
    print(f"Selected tool: {selected_tool}")
    
    # Handle case when no tool is applicable
    if selected_tool == "None":
        return "I don't have a tool that can help with this query."
    
    # STEP 7b: Ask LLM to extract the specific input for the tool
    tool_input_result = tool_input_chain.invoke({
        "selected_tool": selected_tool,
        "query": query
    })
    tool_input = tool_input_result["tool_input"].strip()
    print(f"Tool input: {tool_input}")
    
    # STEP 7c: Execute the selected tool
    tool_output = "Tool not found"
    for tool in tools:
        if tool.name == selected_tool:
            tool_output = tool.func(tool_input)
            break
    print(f"Tool output: {tool_output}")
    
    # STEP 7d: Ask LLM to format a user-friendly response
    response_result = response_chain.invoke({
        "query": query,
        "selected_tool": selected_tool,
        "tool_output": tool_output
    })
    return response_result["response"]


# =============================================================================
# STEP 8: Test with Example Queries
# =============================================================================
def run_examples():
    """Run example queries to demonstrate the manual agent approach."""
    examples = [
        "What is 234 * 78.5?",           # Should use Calculator
        "What's the weather like in San Francisco?",  # Should use WeatherService
        "Tell me about LangChain.",       # Should use Database
        "What's the capital of France?"   # None of the tools can help
    ]
    
    for example in examples:
        print(f"\n\nQUERY: {example}")
        print("-" * 50)
        response = process_query(example)
        print("FINAL RESPONSE:")
        print(response)

In [ ]:
# =============================================================================
# RUN THE EXAMPLES
# =============================================================================
# Execute the examples to see how the manual tool selection works.
# Notice how the LLM correctly identifies which tool to use for each query!

run_examples()



QUERY: What is 234 * 78.5?
--------------------------------------------------
Selected tool: Calculator
Tool input: 234 * 78.5
Tool output: 18369.0
FINAL RESPONSE:
The result of multiplying 234 by 78.5 is 18369.0.


QUERY: What's the weather like in San Francisco?
--------------------------------------------------
Selected tool: WeatherService
Tool input: San Francisco
Tool output: The weather in San Francisco is sunny with a temperature of 72°F.
FINAL RESPONSE:
The current weather in San Francisco is sunny with a temperature of 72°F. It seems like a pleasant day to enjoy outdoor activities or explore the city. Remember to check for any updates in case the weather changes throughout the day.


QUERY: Tell me about LangChain.
--------------------------------------------------
Selected tool: None
FINAL RESPONSE:
I don't have a tool that can help with this query.


QUERY: What's the capital of France?
--------------------------------------------------
Selected tool: None
FINAL RESPONSE: